# Task 1: Housing Price Regression

This task is focused on predicting the price of a house listing based on a pre-determined set of features. This notebook is structured as follows:

1. Setup
2. Exploratory Data Analysis (EDA)
3. Model Evaluation
4. Prediction

# 1. Setup

The following section is required to setup this notebook. Import the appropriate modules and modify model hyperparameters here. 

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.model_selection import train_test_split, PredefinedSplit, GridSearchCV
from sklearn.metrics import mean_squared_error

#import torch
#import torch.nn as nn
#import torch.optim as optim
#from torch.utils.data import Dataset,DataLoader

import re

In [ ]:
# Hyperparameters

RANDOM_CONTROL = 42 # For reproducibility
TRAIN_SIZE = 0.8 # Fraction of training set used for model evaluation; the remaining split is validation set.

# Random Forest: Fill in based on GridSearch results
RF_NUM_ESTIMATORS = 100
RF_MAX_DEPTH = 50
RF_MAX_FEATURES = 1
RF_MIN_SPLIT = 2
RF_MIN_LEAF = 1
RF_BOOTSTRAP = True
RF_CRITERION = "squared_error"

# Gradient Boosting: Fill in based on GridSearch results
GB_NUM_ESTIMATORS = 100
GB_MAX_DEPTH = 50
GB_CRITERION = "squared_error"
GB_LEARNING_RATE = 0.1

# AdaBoost: Fill in based on GridSearch results
AB_NUM_ESTIMATORS = 100
AB_MAX_DEPTH = 50
AB_SPLITTER = "best"
AB_LEARNING_RATE = 0.1

# Extra Trees Regressor: Fill in based on GridSearch results
ET_NUM_ESTIMATORS = 100
ET_MAX_DEPTH = 50
ET_MAX_FEATURES = 1
ET_MIN_SPLIT = 2
ET_MIN_LEAF = 1
ET_BOOTSTRAP = True
ET_CRITERION= "squared_error"

# Bagging Regressor: Fill in based on GridSearch results
BR_NUM_ESTIMATORS = 100
BR_MAX_DEPTH = 50
BR_SPLITTER = "best"
BR_LEARNING_RATE = 0.1

# Neural Net: Fill in based on test iterations
NN_NUM_EPOCHS = 10
NN_BATCH_SIZE = 32
NN_LEARNING_RATE = 0.1

In [ ]:
# Define model training methods here.

# Linear Regression
def train_lr(X_feature, y_label):
    lr = LinearRegression().fit(X_feature, y_label)
    return lr

# Random Forest
def train_rf(X_feature, y_label):
    rf = RandomForestRegressor(n_estimators=RF_NUM_ESTIMATORS, criterion=RF_CRITERION, max_depth=RF_MAX_DEPTH,
                          min_samples_split=RF_MIN_SPLIT, min_samples_leaf=RF_MIN_LEAF, max_features=RF_MAX_FEATURES,
                          bootstrap=RF_BOOTSTRAP, random_state=RANDOM_CONTROL).fit(X_feature, y_label)
    return rf

# Gradient Boosting
def train_gb(X_feature, y_label):
    gb = GradientBoostingRegressor(n_estimators=GB_NUM_ESTIMATORS, learning_rate=GB_LEARNING_RATE,
                                   max_depth=GB_MAX_DEPTH, criterion=GB_CRITERION, 
                                   random_state=RANDOM_CONTROL).fit(X_feature, y_label)
    return gb

# AdaBoost 
def train_ab(X_feature, y_label):
    dt = DecisionTreeRegressor(max_depth=AB_MAX_DEPTH, splitter=AB_SPLITTER,
                               random_state=RANDOM_CONTROL)
    ab = AdaBoostRegressor(base_estimator=dt, n_estimators=AB_NUM_ESTIMATORS, learning_rate=AB_LEARNING_RATE,
                          random_state=RANDOM_CONTROL).fit(X_feature, y_label)
    return ab

# Extra Trees
def train_et(X_feature, y_label):
    et = ExtraTreesRegressor(n_estimators=ET_NUM_ESTIMATORS, criterion=ET_CRITERION, max_depth=ET_MAX_DEPTH,
                          min_samples_split=ET_MIN_SPLIT, min_samples_leaf=ET_MIN_LEAF, max_features=ET_MAX_FEATURES,
                          bootstrap=ET_BOOTSTRAP, random_state=RANDOM_CONTROL).fit(X_feature, y_label)
    return et

# Bagging Regressor
def train_br(X_feature, y_label):
    dt = DecisionTreeRegressor(max_depth=BG_MAX_DEPTH, splitter=BG_SPLITTER,
                               random_state=RANDOM_CONTROL)
    br = BaggingRegressor(base_estimator=dt, n_estimators=BR_NUM_ESTIMATORS, learning_rate=BR_LEARNING_RATE,
                         random_state=RANDOM_CONTROL).fit(X_feature, y_label)
    return br

# Neural Net
def train_nn(X_feature, y_label):
    pass

# Generic predict method for a model.
def predict(model, X_feature) -> pd.DataFrame: # Fix it. Numpy array is returned
    y_hat = model.predict(X_feature)
    return y_hat

# 2. Exploratory Data Analysis (EDA)

We first explore our given training data. This is done via the following sub-tasks:

2a. Visualize
2b. Pre-process
2c. Split
2d. Post-process

## 2a. Visualize

Here, we take our first look at the data to get an idea of how the attribute values are spread.

In [ ]:
# Read training data
df = pd.read_csv('data/train.csv') 
print(df.shape)
df.head()

In [ ]:
def visualize():
    pass

## 2b. Pre-process

Here, we go through the training data and use the information gained in the section above to pick and choose what attributes we wish to consider in our analysis as well as their representation. 

We call this _pre-processing_ as we are working on the whole training set here. To avoid information leakage, we ensure that we do not look at the label data unless required to check for invalid inputs(more on this below). Additionally, we do not perform any aggregation or imputation here to ensure that the validation set does not influence our training set for the following sections.

The _pre-processing_ is comprised of the following tasks which are performed in order:

i.   Remove invalid data
ii.  Remove outliers
iii. Handle missing data
iv.  Transform data
v.   Ignore attributes

TODO: Talk about each attribute here. 

For HDB,

Assume that num_beds refers only to the bedrooms excluding the living rooms
* Hdb 2-room = 1 bedroom , 1 bathroom
* Hdb 3-room = 2 bedroom , 2 bathroom
* Hdb 4-room = 3 bedroom , 2 bathroom
* Hdb 5-room = 3 bedroom , 2 bathroom
* Hdb 3-gen = 4 bedroom , 3 bathroom
* Hdb Executive = 3/4 bedroom, 2/3 bathroom
* Hdb Masionette = 3/4 bedroom, 2/3 bathroom
* Hdb Jumbo = 4 bedroom, 4 bathroom

References:
* http://www.data.com.sg/template-m.jsp?p=my/1.html 
* https://www.hdb.gov.sg/residential/buying-a-flat/finding-a-flat/types-of-flats


In [ ]:
def remove_invalid_values(df) -> pd.DataFrame:
    # Price is the target regression variable. If negative or 0, treat that row as invalid (logical assumption)
    if 'price' in df:
        df = df[df.price > 0]

    # Filter out HDB prices more than 2,000,000 (real-world knowledge; ref. source)
    if 'price' in df:
        filter_price_hdb = ((df.price > 2000000) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) | (df.title.str.contains('hdb','Hdb', regex=False))))
        df = df.drop(df[filter_price_hdb].index)
        
    # Filter out HDB with more bathrooms than bedrooms (real-world knowledge; ref. source)
    filter_bath_beds_hdb = ((df.num_baths > df.num_beds) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) | (df.title.str.contains('hdb','Hdb', regex=False))))
    df = df.drop(df[filter_bath_beds_hdb].index)

    # Filter out HDB with more than 4 bathrooms or 6 bedrooms (real-world knowledge; ref. source)
    filter_bath_beds_4_hdb = (((df.num_baths > 4) | (df.num_beds > 6)) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) | (df.title.str.contains('hdb','Hdb', regex=False))))
    df = df.drop(df[filter_bath_beds_4_hdb].index)
    
        
    return df

def remove_outliers(df) -> pd.DataFrame:
    # Filter out HDB with size > 2000 sqft (outlier detection)
    filter_size_hdb = ((df.size_sqft > 2000) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) | (df.title.str.contains('hdb','Hdb', regex=False))))
    df[filter_size_hdb][['property_type','num_baths','num_beds','size_sqft','price']]
    df = df.drop(df[filter_size_hdb].index)
    
    # Target label is used below for outlier detection. Appropriately tagged data is dropped entirely.
    if 'price' in df:
    # Filtering those data with less than $200/square feet
        df['price per sq ft'] = df['price']/df['size_sqft']
        filter_price_sqft_200 = ((df['price per sq ft'] < 200) & (df['price per sq ft'] > 0))
        df = df.drop(df[filter_price_sqft_200].index)
    # Filtering those data with less than 500 square feet, and more than $5000 per square feet
        df['price per sq ft'] = df['price']/df['size_sqft']
        filter_size = ((df['price per sq ft'] > 5000) & (df['price per sq ft'] > 0) & (df['size_sqft'] < 500))
        df = df.drop(df[filter_size].index)
        
        df.drop('price per sq ft', axis=1, inplace=True)
    
    return df

def handle_missing_values(df) -> pd.DataFrame:
    # Treat missing year data as new.
    # Semantically, we define this attribute as the depreciation factor for pricing.
    # A new house or one with missing data denotes the depreciation factor is 0 or unknown.
    # The depreciation factor is assumed to be the difference between construction and current year.
    # TODO: Maybe do not treat future years as current! Inflation factor might be one to look out for.
    df['built_year'] = df['built_year'].fillna(2022)
    
    # TODO: 80 are missing. Should we remove them or should we keep it as 0?
    # Verify assumption if studio qualifies as 1 bed. 
    # 75 of missing are studio, we replace the Nan as 1
    filter_beds_studio = ((df.num_beds.isna()) & ((df.title.str.contains('studio','Studio', flags=re.IGNORECASE, regex=True))))
    df.loc[filter_beds_studio, "num_beds"] = 1
    # 5 of missing, we do not have much info. Use 0 to denote absence of attribute
    df['num_beds'] = df['num_beds'].fillna(0)

    # TODO: 400 are missing. Cannot remove so many data. Use 0 to denote absence of attribute.
    df['num_baths'] = df['num_baths'].fillna(0)
    
    
    return df

def transform_data(df) -> pd.DataFrame:
    # TODO: Test against not doing this.
    df.loc[df["built_year"] > 2022, "built_year"] = 2022
    
    # Convert built_year into the aforementioned depreciation factor
    df["depreciation"] = (2022-df["built_year"])
    '''
    # TODO: Add details on why we are doing so
    df.loc[df.property_type.str.contains('hdb', flags=re.IGNORECASE, regex=True), 'property_type'] = 'hdb'
    df.loc[df.property_type.str.contains('condo', flags=re.IGNORECASE, regex=True), 'property_type'] = 'condo'
    df['property_type'] = df['property_type'].str.lower()
    df.loc[df.property_type.str.contains('cluster house', flags=re.IGNORECASE, regex=True), 'property_type'] = 'landed'
    df.loc[df.property_type.str.contains('townhouse', flags=re.IGNORECASE, regex=True), 'property_type'] = 'landed'
    df.loc[df.property_type.str.contains('land only', flags=re.IGNORECASE, regex=True), 'property_type'] = 'landed'
    df.loc[df.property_type.str.contains('apartment',  flags=re.IGNORECASE, regex=True), 'property_type'] = 'condo'
    df.loc[df.property_type.str.contains('bungalow', flags=re.IGNORECASE, regex=True), 'property_type'] = 'bungalow'
    df.loc[df.property_type.str.contains('semi-detached house', flags=re.IGNORECASE, regex=True), 'property_type'] = 'corner'
    df.loc[df.property_type.str.contains('corner terrace',flags=re.IGNORECASE, regex=True), 'property_type'] = 'corner'
    df.loc[df.property_type.str.contains('shophouse', flags=re.IGNORECASE, regex=True), 'property_type'] = 'protected'
    df.loc[df.property_type.str.contains('conservation house', flags=re.IGNORECASE, regex=True), 'property_type'] = 'protected'
    
    # Get one hot encoding of columns property_type
    one_hot = pd.get_dummies(df['property_type'])
    # Join the encoded df
    df = df.join(one_hot)
    '''
    df['tenure'] = df['tenure'].fillna(value=df.property_type)
    df.loc[df.tenure.str.contains('hdb', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('condo', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('terraced house', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('corner', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('landed', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('protected', flags=re.IGNORECASE, regex=True), 'tenure'] = 'freehold'
    df.loc[df.tenure.str.contains('bungalow', flags=re.IGNORECASE, regex=True), 'tenure'] = 'freehold'

    df.loc[df.tenure.str.contains('110-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('103-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('102-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('100-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'

    df.loc[df.tenure.str.contains('999-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = 'freehold'
    df.loc[df.tenure.str.contains('946-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = 'freehold'
    df.loc[df.tenure.str.contains('956-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = 'freehold'
    df.loc[df.tenure.str.contains('947-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = 'freehold'
    df.loc[df.tenure.str.contains('929-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = 'freehold'

    df['encoded_tenure'] = 0
    df.loc[df.tenure.str.contains('freehold', flags=re.IGNORECASE, regex=True), 'encoded_tenure'] = 1
    
    return df

def ignore_attributes(df) -> pd.DataFrame:
    # Drop listing id; nominal identifier with no meaning
    df.drop('listing_id', axis=1, inplace=True)

    # Drop elevation; all the values are 0, spurious attribute
    df.drop('elevation', axis=1, inplace=True)

    # Drop url; nominal identifier with no meaning; useful for manual lookups or scraping
    df.drop('property_details_url', axis=1, inplace=True)

    # Drop floor level as 83% missing and sparse with the rest of the values. 
    # Not enough data available to get the model trained.
    df.drop('floor_level', axis=1, inplace=True)

    # Drop column property_type and tenure as it is now encoded
    df.drop('property_type',axis = 1, inplace=True)
    df.drop('tenure',axis=1, inplace=True)

    # BELOW CODE IN THIS SECTION IS ONLY MEANT TO GET THE SKELETON WORKING; RE-EVALUATE EACH ATTRIBUTE ONE BY ONE
    df.drop('title', axis=1, inplace=True)
    df.drop('address', axis=1, inplace=True)
    df.drop('property_name', axis=1, inplace=True)
    df.drop('built_year', axis=1, inplace=True)
    df.drop('furnishing', axis=1, inplace=True)
    df.drop('available_unit_types', axis=1, inplace=True)
    df.drop('total_num_units', axis=1, inplace=True) # Bernard verified dropping it. 27.9% missing.
    df.drop('lat', axis=1, inplace=True)
    df.drop('lng', axis=1, inplace=True)
    df.drop('subzone', axis=1, inplace=True)
    df.drop('planning_area', axis=1, inplace=True)

    return df

def pre_process(df, mode='train') -> pd.DataFrame:
    if mode=='train': 
        df = remove_invalid_values(df)
        df = remove_outliers(df)
    df = handle_missing_values(df)
    df = transform_data(df)
    df = ignore_attributes(df)
    
    return df

In [ ]:
df_preprocessed = pre_process(df, mode='train')
print(df_preprocessed.shape)
df_preprocessed.head()

## 2c. Split

Here, we split the training data into training and validation sets, based on the user-defined factor from hyperparameters above. This is done solely to evaluate each model's performance as it is trained on the split set and tested against the validation set. Since it is a regression task, we use RMSE as our performance metric to compare predictions against target labels. Lower values are better.

In [ ]:
# Split data into train and validation set

X_housing = df_preprocessed.loc[:, df_preprocessed.columns != 'price']
y_housing = df_preprocessed['price']

X_train, X_val, y_train, y_val = train_test_split(X_housing, y_housing, train_size=TRAIN_SIZE, random_state=RANDOM_CONTROL, shuffle=True) 

# print(X_train.shape)
# print(y_train.shape)


## 2d. Post-process

Here, we separately process the training and validation sets. Since we perform aggregation-based methods here, this is done after split to avoid information leakage and ensures salient evaluation results.

If you wish to explore each model and go through our data handling, proceed to Section 3 - Model Evaluation. Alternately, if you are only interested in the final output, you may jump to Section 4 - Prediction to gather results.

In [ ]:
# Standardize/Normalize

def post_process(df) -> pd.DataFrame:
    return df


X_train = post_process(X_train)
X_val = post_process(X_val)

# 3. Model Evaluation

Here, we evaluate models for their predictions. RMSE is the metric used for comparison. The following models are evaluated:

3a. Linear Regression
3b. Random Forest
3c. Gradient Boosting
3d. AdaBoost
3e. Extra Trees Regressor
3f. Bagging Regressor
3g. Neural Net

This section also involves hyperparameter tuning wherein we search for best settings per model. Those settings are later used to set hyperparameters as in Section 1 and dictate the final predictions in Section 4. 

In [ ]:
def validate(y, y_hat) -> None:
    rmse = mean_squared_error(y, y_hat, squared=False)
    print('Validation RMSE: {:.3}'.format(rmse))
    return

# Use our predefined split for GridSearch below.
val_split_indices = [-1 if x in X_train.index else 0 for x in X_housing.index]
ps = PredefinedSplit(test_fold=val_split_indices)

## 3a. Linear Regression

In [ ]:
model_lr = train_lr(X_train, y_train)
y_hat_lr = predict(model_lr, X_val)
validate(y_val, y_hat_lr)

## 3b. Random Forest

In [ ]:
def bestfit_rf(X_feature, y_label, train_test_split):
    estimator = RandomForestRegressor()
    params = {'n_estimators': [25, 50, 100],
              'max_depth': [5, 10, 25, 50],
              'min_samples_split': [2],
              'min_samples_leaf': [1],
              'criterion': ["squared_error"],
              'max_features': [1],
              'bootstrap': [True, False],
              'random_state': [RANDOM_CONTROL]}
    model_rf = GridSearchCV(estimator=estimator,
                         param_grid=params,
                         cv=train_test_split)
    model_rf.fit(X_feature, y_label)
    print('Random Forest Best Parameters: {}'.format(model_rf.best_params_))
    return model_rf

model_rf = bestfit_rf(X_housing, y_housing, ps)
y_hat_rf = predict(model_rf, X_val)
validate(y_val, y_hat_rf)

## 3c. Gradient Boosting

In [ ]:
def bestfit_gb(X_feature, y_label, train_test_split):
    estimator = GradientBoostingRegressor()
    params = {'n_estimators': [25, 50, 100],
              'learning_rate': [0.001, 0.01, 0.1, 0.5, 1],
              'max_depth': [5, 10, 25, 50],
              'min_samples_split': [2],
              'min_samples_leaf': [1],
              'criterion': ["squared_error", "friedman_mse"],
              'max_features': [1],
              'random_state': [RANDOM_CONTROL]}
    model_gb = GridSearchCV(estimator=estimator,
                         param_grid=params,
                         cv=train_test_split)
    model_gb.fit(X_feature, y_label)
    print('Gradient Boost Best Parameters: {}'.format(model_gb.best_params_))
    return model_gb
    

model_gb = bestfit_gb(X_housing, y_housing, ps)
y_hat_gb = predict(model_gb, X_val)
validate(y_val, y_hat_gb)

## 3d. AdaBoost

In [ ]:
def bestfit_ab(X_feature, y_label, train_test_split):
    base_estimator = DecisionTreeRegressor()
    estimator = AdaBoostRegressor(base_estimator=base_estimator)
    params = {'base_estimator__max_depth': [5, 10, 25, 50],
              'base_estimator__splitter': ['best', 'random'],
              'n_estimators': [25, 50, 100],
              'learning_rate': [0.001, 0.01, 0.1, 0.5, 1],
              'random_state': [RANDOM_CONTROL]}
    model_ab = GridSearchCV(estimator=estimator,
                         param_grid=params,
                         cv=train_test_split)
    model_ab.fit(X_housing, y_housing)
    print('AdaBoost Best Parameters: {}'.format(model_ab.best_params_))
    return model_ab


model_ab = bestfit_ab(X_housing, y_housing, ps)
y_hat_ab = predict(model_ab, X_val)
validate(y_val, y_hat_ab)

## 3e. Extra Trees Regressor

In [ ]:
def bestfit_et(X_feature, y_label, train_test_split):
    estimator = ExtraTreesRegressor()
    params = {'n_estimators': [25, 50, 100],
              'max_depth': [5, 10, 25, 50],
              'min_samples_split': [2],
              'min_samples_leaf': [1],
              'criterion': ["squared_error", "friedman_mse"],
              'max_features': [1],
              'bootstrap': [True, False],
              'random_state': [RANDOM_CONTROL]}
    model_et = GridSearchCV(estimator=estimator,
                         param_grid=params,
                         cv=train_test_split)
    model_et.fit(X_housing, y_housing)
    print('Extra Trees Best Parameters: {}'.format(model_et.best_params_))
    return model_et


model_et = bestfit_et(X_housing, y_housing, ps)
y_hat_et = predict(model_et, X_val)
validate(y_val, y_hat_et)

## 3f. Bagging Regressor

In [ ]:
def bestfit_br(X_feature, y_label, train_test_split):
    base_estimator = DecisionTreeRegressor()
    estimator = BaggingRegressor(base_estimator=base_estimator)
    params = {'n_estimators': [25, 50, 100],
              'max_features': [1],
              'random_state': [RANDOM_CONTROL]}
    model_br = GridSearchCV(estimator=estimator,
                         param_grid=params,
                         cv=ps)
    model_br.fit(X_housing, y_housing)
    print('Bagging Best Parameters: {}'.format(model_br.best_params_))
    return model_br


model_br = bestfit_br(X_housing, y_housing, ps)
y_hat_br = predict(model_br, X_val)
validate(y_val, y_hat_br)

## 3g. Neural Net

In [ ]:
X_housing = X_housing.to_numpy()
y_housing = y_housing.to_numpy()
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_val = X_val.to_numpy()
y_val = y_val.to_numpy()

#print(X_train.shape)
#print(y_train.shape)
train_dataloader = DataLoader([ [X_train[i], y_train[i]] for i in range(len(X_train)) ], batch_size=NN_BATCH_SIZE, shuffle=True, num_workers=4)

In [ ]:
# Define MLP architecture
class Model(nn.Module):
    
    def __init__(self, device='cpu'):
        super(Model, self).__init__()
        self.device = device
        
        # Modify accordingly
        self.fc1 = nn.Linear(1, 4)
        self.fc2 = nn.Linear(4, 16)
        self.fc3 = nn.Linear(16, 64)
        self.fc4 = nn.Linear(64, 4)
        self.fc5 = nn.Linear(4, 1)
        self.relu = nn.ReLU()
        
        self.dense = nn.Sequential(self.fc1, self.relu, self.fc2, self.relu, self.fc3, self.relu, 
                                   self.fc4, self.relu, self.fc5)
        
    def forward(self, x):
        pred = self.dense(x)
        return pred
    
# Train
device = 'cpu'
model = Model()
optimizer = optim.Adam(model.parameters(), NN_LEARNING_RATE)
criterion = nn.MSELoss()
model.to(device)
for epoch in range(NN_NUM_EPOCHS):
    running_loss = 0
    for idx, (x_features, y_labels) in enumerate(train_dataloader):
        optimizer.zero_grad()
        x_features = x_features.to(device, dtype=torch.float)
        y_labels = y_labels.to(device, dtype=torch.float)
        prediction = model(x_features)
        loss = torch.sqrt(criterion(prediction, y_labels)) # Standardize RMSE loss
        loss.backward()
        optimizer.step()
        running_loss += loss
        if (idx+1) %100 == 0: 
            running_loss = format(running_loss/100, '.4f')
            print(f"Epoch [{epoch+1} Batches processed | {idx}] Loss: {running_loss}")
            running_loss = 0
print("Finished Training.")


In [ ]:
# Validate
X_features = torch.from_numpy(X_val)
y_labels = torch.from_numpy(y_val)
X_features = X_features.to(device, dtype=torch.float)
y_labels = y_labels.to(device, dtype=torch.float)
prediction = model(X_features)
rmse_nn = torch.sqrt(criterion(prediction, y_labels))

print('Neural Net Validation rmse: {:.3}'.format(rmse_nn))

# 4. Prediction

Now that we are done evaluating each model and comparing their performance, we select the model that performed best on the validation set to generate our overall predictions. We re-train this model from scratch on the whole set of training data here as it is assumed ready to ship so we wish to use as much data for training as we can. We later test this model against the provided test data and generate a CSV file of predicted values.

The following identifiers are used for each model - 'lr' for linear regression, 'rf' for random forest, 'gb' for gradient boosting, 'ab' for adaboost, 'et' for extra trees regressor, 'br' for bagging regressor, 'nn' for neural net.

In [ ]:
# Train over entire training set
df_train = pd.read_csv('data/train.csv') 
df_train = pre_process(df_train, mode='train')
X_train = df_train.loc[:, df_train.columns != 'price']
y_train = df_train['price']
X_train = post_process(X_train)
model = train_rf(X_train, y_train) # Choose your model here. Current choice: 'rf' for Random Forest

# Predict labels for test set
df_test = pd.read_csv('data/test.csv') 
df_test = pre_process(df_test, mode='test')
df_test = post_process(df_test)
predictions = predict(model, df_test)
predictions = pd.DataFrame(predictions)
predictions.to_csv('predictions.csv', header=['Predicted'], index=True, index_label='Id')